<a href="https://colab.research.google.com/github/themysterysolver/FYP-26/blob/main/CODE/DS_1000.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DS 1000

- Data science 1000

#### PIP NSTALL

In [1]:
#I ain't doing this and resolving the error in below block!
!pip install datasets
!pip install transformers datasets accelerate torch
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 16.4 MB/s eta 0:00:00


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import sys
import re

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### DATA LOADING

In [4]:
from datasets import load_dataset
ds1k = load_dataset("xlangai/DS-1000")
df_ds1k = ds1k['test'].to_pandas()

README.md:   0%|          | 0.00/554 [00:00<?, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
# Use the .str accessor to treat the column as a series of strings
df_ds1k['prompt_2'] = df_ds1k['prompt'].str.split('A:', n=1).str[0].str.strip()

# Verification
print(df_ds1k['prompt_2'].iloc[0])

Problem:
I have the following DataFrame:
    Col1  Col2  Col3  Type
0      1     2     3     1
1      4     5     6     1
2      7     8     9     2
3    10    11    12     2
4    13    14    15     3
5    16    17    18     3


The DataFrame is read from a CSV file. All rows which have Type 1 are on top, followed by the rows with Type 2, followed by the rows with Type 3, etc.
I would like to shuffle the order of the DataFrame's rows according to a list. \
For example, give a list [2, 4, 0, 3, 1, 5] and desired result should be:
    Col1  Col2  Col3  Type
2      7     8     9     2
4     13    14    15     3
0     1     2     3     1
3    10    11    12     2
1     4     5     6     1
5    16    17    18     3
...


How can I achieve this?


In [6]:
print(df_ds1k['prompt_2'][876])

Problem:

Given a list of variant length features, for example:

f = [
    ['t1'],
    ['t2', 't5', 't7'],
    ['t1', 't2', 't3', 't4', 't5'],
    ['t4', 't5', 't6']
]
where each sample has variant number of features and the feature dtype is str and already one hot.

In order to use feature selection utilities of sklearn, I have to convert the features to a 2D-array which looks like:

f
    t1  t2  t3  t4  t5  t6  t7
r1   1   0   0   0   0   0   0
r2   0   1   0   0   1   0   1
r3   1   1   1   1   1   0   0
r4   0   0   0   1   1   1   0
How could I achieve it via sklearn or numpy?


### MODEL LOADING


In [ ]:
#I ain't doing this and resolving the error in below block!
!pip install datasets
!pip install transformers datasets accelerate torch
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 10.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import sys
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:

model_id = "Qwen/Qwen2.5-Coder-3B-Instruct"

# 1. Setup 4-bit quantization (Standard Hugging Face way)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 2. Load Tokenizer and Model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically puts it on the GPU
    low_cpu_mem_usage=True # Prevents the System RAM crash
)

print(f"✅ Success! {model_id} loaded using standard transformers.")

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/434 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

✅ Success! Qwen/Qwen2.5-Coder-3B-Instruct loaded using standard transformers.


### FUNCTIONS

#### run_ds1000_official_test

In [8]:
def run_ds1000_official_test(clean_code, code_context):
    """
    Executes the code_context to load the official test_execution function,
    then runs the generated code through it.
    """
    # Create a sandbox environment
    test_env = {}

    try:

        exec(code_context, test_env)
        test_env['test_execution'](clean_code)

        return "Passed"

    except AssertionError:
        return "Failed: Logic Error (Assertion failed)"
    except Exception as e:
        # Catch syntax errors in model code or execution issues
        return f"Failed: {type(e).__name__}: {str(e)}"

In [9]:
# def prompt_constuction(raw_prompt, exec_context_snippet):
#     """
#     Constructs a prompt that forces the model to perform the specific task
#     and store the final outcome in the 'result' variable for automated testing.
#     """
#     system_message = (
#         "You are an expert Data Scientist specializing in Python and library-specific "
#         "tasks (Pandas, NumPy, Scikit-learn). Your output must be valid code only."
#     )

#     user_content = (
#         "### ENVIRONMENT CONTEXT\n"
#         "The following code is already defined in the environment. Do NOT redefine these variables:\n"
#         f"```python\n{exec_context_snippet}\n```\n\n"
#         "### PROBLEM SPECIFICATION\n"
#         f"{raw_prompt}\n\n"
#         "### MANDATORY CONSTRAINTS\n"
#         "1. **Imports**: Only import libraries not already provided in the context.\n"
#         "2. **Logic**: Write the Python logic required to solve the specification above.\n"
#         "3. **Final Storage**: You MUST store the final answer/object in a variable named `result`.\n"
#         "4. **No Output**: Do not use `print()`. Do not include explanations.\n"
#         "5. **Format**: Make sure the fina output starts with ENVIRONEMNT CONTEXT and along with any libraries if needed!,then continue with coding. text_input is a must field."
#     )

#     messages = [
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": user_content}
#     ]

#     return messages

In [10]:
def construct_ds1000_prompt_final(raw_prompt, exec_context_snippet):
    system_message = (
        "You are an expert Data Scientist. You write clean, functional Python code. "
        "Your responses consist ONLY of Python code blocks."
    )

    user_content = (
        "### AVAILABLE CONTEXT\n"
        "The following variables and libraries are already initialized:\n"
        f"```python\n{exec_context_snippet}\n```\n\n"
        "### TASK\n"
        f"Problem: {raw_prompt}\n\n"
        "### REQUIREMENTS\n"
        "1. Use the variables provided in the context above (specifically `test_input`).\n"
        "2. Do NOT redefine `test_input`, `df`, or `List`.\n"
        "3. You must perform the required logic and store the final output in a variable named `result`.\n"
        "4. Output ONLY the code logic. No explanations, no markdown intro, just the Python code."
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

#### construct_ds1000_prompt_final_x

In [11]:
def construct_ds1000_prompt_final_x(raw_prompt, exec_context_snippet):
    """
    Constructs a structured prompt for DS-1000 tasks, ensuring
    the model outputs code that can be directly executed
    within the provided context.
    """
    system_message = (
        "You are an expert Data Scientist. You solve complex data manipulation "
        "tasks using Python. You respond exclusively with functional Python code "
        "wrapped in a markdown block. Do not provide explanations or commentary."
    )

    user_content = (
        "### ENVIRONMENT CONTEXT\n"
        "The following code is already executed. Do not redefine these variables:\n"
        f"```python\n{exec_context_snippet}\n```\n\n"
        "### PROBLEM SPECIFICATION\n"
        f"{raw_prompt}\n\n"
        "### INSTRUCTIONS\n"
        "1. Write the Python code to solve the problem above.\n"
        "2. Ensure you use the existing variables (like `test_input` or `df`) as defined.\n"
        "3. Your code must culminate in assigning the final answer to a variable named `result`.\n"
        "4. Output ONLY the code logic. No text before or after the code block."
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

In [12]:
def construct_ds1000_prompt_final_v2(raw_prompt, exec_context_snippet):
    system_message = (
        "You are an expert Data Scientist. You write clean, functional Python code. "
        "Your responses consist ONLY of Python code blocks."
    )

    user_content = (
        "### AVAILABLE CONTEXT\n"
        "The following variables and libraries are already initialized:\n"
        f"```python\n{exec_context_snippet}\n```\n\n"
        "### TASK\n"
        f"Problem: {raw_prompt}\n\n"
        "### REQUIREMENTS\n"
        "1. Understand the above TASK and complete the AVAILABLE CONTEXT.\n"
        "2. Do NOT redefine `test_input`, `df`, or `List` make sure the AVAILABLE CONTEXT is INTACT.\n"
        "3. If u need any Libararies u can import just beofre the AVAILABLE CONTEXT. Don't use the code from TASK it is just for your understadning."
        "4. You must perform the required logic and store the final output in a variable named `result`.\n"
        "5. Output ONLY the code logic. No explanations, no markdown intro, just the Python code."
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

In [13]:
def construct_ds1000_prompt_final_v3(raw_prompt, exec_context_snippet):
    system_message = (
        "You are an expert Data Scientist. You complete partial Python scripts. "
        "Your output must be ONLY valid Python code that integrates with a specific environment."
    )

    user_content = (
        "### 1. LIVE ENVIRONMENT (MANDATORY)\n"
        "Your code will be executed IMMEDIATELY AFTER this block. These variables ALREADY EXIST:\n"
        f"```python\n{exec_context_snippet}\n```\n\n"

        "### 2. PROBLEM DESCRIPTION (FOR UNDERSTANDING ONLY)\n"
        "Use this description to understand the logic required. Do NOT copy code snippets or dummy data from here:\n"
        f"--- START DESCRIPTION ---\n{raw_prompt}\n--- END DESCRIPTION ---\n\n"

        "### 3. STRICT REQUIREMENTS\n"
        "- **Context Continuity**: Your code must use the variable `test_input` from the LIVE ENVIRONMENT above.\n"
        "- **No Redefinition**: Do NOT re-initialize `df`, `List`, or `test_input`. Assume they are already filled with data.\n"
        "- **Final Output**: Assign the result of your logic to a variable named `result`.\n"
        "- **Zero Explanation**: Provide ONLY the code. Do not include triple backticks or conversational text."
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

#### Extract_only_exec_context

In [14]:
def extract_only_exec_context(code_context):
    """
    Specifically extracts the raw string assigned to exec_context.
    This contains only the imports and variable mappings used in the test.
    """
    # Look for the raw string pattern: exec_context = r""" ... """
    pattern = r'exec_context = r"""(.*?)"""'
    match = re.search(pattern, code_context, re.DOTALL)

    if match:
        content = match.group(1).strip()
        # We leave the [insert] marker out of the prompt snippet
        # so the model doesn't get confused by the tag itself.
        return content.replace("[insert]", "").strip()

    return "import pandas as pd\nimport numpy as np" # Basic fallback

In [49]:
def extract_only_exec_context_wi(code_context):
    """
    Specifically extracts the raw string assigned to exec_context.
    This contains only the imports and variable mappings used in the test.
    Along with the insert marker.
    """
    # Look for the raw string pattern: exec_context = r""" ... """
    pattern = r'exec_context = r"""(.*?)"""'
    match = re.search(pattern, code_context, re.DOTALL)

    if match:
        content = match.group(1).strip()
        # We leave the [insert] marker out of the prompt snippet
        # so the model doesn't get confused by the tag itself.
        return content.strip()

    return "import pandas as pd\nimport numpy as np" # Basic fallback

- while executing I found out theat [INSERT] is always missing.

In [48]:
for i in range(25):
  print(extract_only_exec_context(df_ds1k['code_context'][i]))
  print('-------')
  print(df_ds1k['code_context'][i])
  print('----------------------------------')

import pandas as pd
import numpy as np
df, List = test_input
-------
import pandas as pd
import numpy as np
import copy


def generate_test_case(test_case_id):
    def generate_ans(data):
        data = data
        df, List = data
        return df.iloc[List]

    def define_test_input(test_case_id):
        if test_case_id == 1:
            df = pd.DataFrame(
                {
                    "Col1": [1, 4, 7, 10, 13, 16],
                    "Col2": [2, 5, 8, 11, 14, 17],
                    "Col3": [3, 6, 9, 12, 15, 18],
                    "Type": [1, 1, 2, 2, 3, 3],
                }
            )
            List = np.random.permutation(len(df))
        return df, List

    test_input = define_test_input(test_case_id)
    expected_result = generate_ans(copy.deepcopy(test_input))
    return test_input, expected_result


def exec_test(result, ans):
    try:
        pd.testing.assert_frame_equal(result, ans, check_dtype=False)
        return 1
    except:
        return 0


exe

In [50]:
for i in range(25):
  print(extract_only_exec_context_wi(df_ds1k['code_context'][i]))
  # print('-------')
  # print(df_ds1k['code_context'][i])
  print('----------------------------------')

import pandas as pd
import numpy as np
df, List = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
df, List = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
df = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
df = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
def f(df):
[insert]
df = test_input
result = f(df)
----------------------------------
import pandas as pd
import numpy as np
df = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
df = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
df = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
df = test_input
[insert]
----------------------------------
import pandas as pd
import numpy as np
df = test_input
[insert]
-------

In [15]:
#print(df_ds1k['prompt'][0])

In [16]:
#print(df_ds1k['code_context'][0])

In [17]:
print(extract_only_exec_context(df_ds1k['code_context'][0]))

import pandas as pd
import numpy as np
df, List = test_input


In [18]:
construct_ds1000_prompt_final

<function __main__.construct_ds1000_prompt_final(raw_prompt, exec_context_snippet)>

#### constuct prompt ds1k

In [19]:
def contruct_prompt_ds1k(raw_prompt, exec_context_snippet):
    """
    Constructs a DS-1000–compatible prompt where the model generates
    an INSERTABLE code fragment (not a full program).
    """

    system_message = (
        "You are a senior Python data scientist.\n"
        "You generate ONLY executable Python code fragments.\n"
        "You NEVER explain, comment, or add markdown outside the code.\n"
        "Your output will be inserted into an existing execution context."
    )

    user_message = (
        "You are given a Python execution environment where some variables "
        "and imports ALREADY EXIST.\n\n"

        "===== EXECUTION CONTEXT (ALREADY EXECUTED) =====\n"
        "The following code has already been run. You MUST NOT redefine these variables.\n"
        "You MUST CONTINUE from this state:\n\n"
        f"{exec_context_snippet}\n\n"

        "===== TASK =====\n"
        f"{raw_prompt}\n\n"

        "===== STRICT INSERTION RULES =====\n"
        "Your output will replace a placeholder named [insert].\n"
        "Therefore:\n"
        "- Do NOT redefine input variables (e.g., test_input, f, df).\n"
        "- You MAY import libraries IF needed (imports MUST be at the top).\n"
        "- Use ONLY variables that already exist in the environment.\n"
        "- Perform the required computation.\n"
        "- Assign the final output to a variable named `result`.\n"
        "- Do NOT define functions or classes unless absolutely required.\n"
        "- Do NOT print anything.\n"
        "- Do NOT include markdown, comments, or explanations.\n\n"

        "===== OUTPUT FORMAT (MANDATORY) =====\n"
        "Return ONLY raw Python code.\n"
        "The code MUST be directly insertable at [insert] without modification.\n"
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]


##### contruct_prompt_ds1k_v2

- overcoming the prob at indentadtion and result
- making sure it's an insert.

In [53]:
def contruct_prompt_ds1k_v2(raw_prompt, exec_context_snippet):
    """
    Constructs a DS-1000–compatible prompt where the model generates
    a STRICTLY INSERTABLE code fragment that completes surrounding code.
    """

    system_message = (
        "You are a senior Python data scientist.\n"
        "You generate ONLY Python code fragments meant to be INSERTED into existing code.\n"
        "You NEVER explain, comment, or wrap output in markdown.\n"
        "You strictly obey the insertion contract."
    )

    user_message = (
        "You are working inside an EXISTING Python program.\n"
        "Your output will be INSERTED verbatim at a placeholder named [insert].\n\n"

        "===== EXECUTION CONTEXT (DO NOT MODIFY) =====\n"
        "This code exists BEFORE and AFTER the insertion point.\n"
        "You MUST reason about both sides when generating code.\n\n"
        f"{exec_context_snippet}\n\n"

        "===== TASK =====\n"
        f"{raw_prompt}\n\n"

        "===== INSERTION CONTRACT (CRITICAL) =====\n"
        "- Your code will replace [insert] exactly.\n"
        "- Code BEFORE [insert] may define variables, control flow, or partially defined functions.\n"
        "- Code AFTER [insert] may depend on variables or functions you complete.\n"
        "- You MUST complete any open blocks (functions, loops, conditionals) if applicable.\n"
        "- You MUST use correct indentation relative to surrounding code.\n"
        "- Do NOT redefine variables that already exist.\n"
        "- Do NOT redefine functions that already exist.\n"
        "- Do NOT assume `result` must be assigned unless explicitly required by the task.\n"
        "- Modify or finalize existing variables so that the code AFTER [insert] works correctly.\n"
        "- You MAY add imports ONLY if strictly necessary.\n"
        "- Do NOT print anything.\n"
        "- Do NOT include comments or explanations.\n\n"

        "===== OUTPUT FORMAT =====\n"
        "Return ONLY raw Python code that can be directly inserted at [insert].\n"
        "Your output must be syntactically and semantically compatible with the surrounding code.\n"
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]


#### def contruct_prompt_ds1k_v3


In [57]:
def contruct_prompt_ds1k_v3(raw_prompt, exec_context_snippet):
    """
    Simple DS-1000 prompt: minimal code, minimal hallucination.
    """

    system_message = (
        "You are a Python data scientist.\n"
        "You write concise, correct Python code for data manipulation.\n"
        "You prefer direct, vectorized solutions over complex logic."
    )

    user_message = (
        "You are given a Python code snippet with a placeholder [insert].\n"
        "Your code will be INSERTED at that position.\n\n"

        "===== EXISTING CODE =====\n"
        f"{exec_context_snippet}\n\n"

        "===== TASK =====\n"
        f"{raw_prompt}\n\n"

        "===== GUIDELINES =====\n"
        "- Write the simplest correct solution.\n"
        "- Prefer short, direct Pandas / NumPy operations.\n"
        "- Do NOT define helper functions or classes.\n"
        "- Do NOT print anything.\n"
        "- You MAY add imports if needed.\n"
        "- Use existing variables from the context.\n"
        "- Ensure the final output is available in variable `result`.\n\n"

        "===== OUTPUT =====\n"
        "Return ONLY raw Python code.\n"
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]


In [59]:
def contruct_prompt_ds1k_v4(raw_prompt, exec_context_snippet):
    """
    Simple DS-1000 prompt: minimal code, minimal hallucination.
    """

    system_message = (
        "You are a Python data scientist.\n"
        "You write concise, correct Python code for data manipulation.\n"
        "You prefer direct, vectorized solutions over complex logic."
    )

    user_message = (
        "You are given a Python code snippet with a placeholder [insert].\n"
        "Your code will be INSERTED at that position.\n\n"

        "===== EXISTING CODE =====\n"
        f"{exec_context_snippet}\n\n"

        "===== TASK =====\n"
        f"{raw_prompt}\n\n"

        "===== GUIDELINES =====\n"
        "- Think deeply about the given TASK before coding.\n"
        "- Write the simplest correct solution.\n"
        "- Prefer short, direct Pandas / NumPy operations.\n"
        "- Do NOT define helper functions or classes.\n"
        "- Do NOT print anything.\n"
        "- You MAY add imports if needed.\n"
        "- Use existing variables from the context.\n"
        "- Make sure `result` variable is declared before using it.\n"
        "- Give proper intendation at [INSERT] if the line before [INSERT] is a function.\n"
        "- Ensure the final output is available in variable `result`.This is really important.\n\n"

        "===== OUTPUT =====\n"
        "Return ONLY raw Python code.\n"
    )

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message}
    ]


In [20]:
# def run_test(clean_code, code_context):
#     """
#     Executes the code_context to load the official test_execution function,
#     then runs the generated code through it.
#     """
#     # Create a sandbox environment
#     test_env = {}
#     test_env_2 ={}
#     try:
#         try:
#           exec(code_context, test_env)
#           generate_test_case = test_env['generate_test_case']
#           exec_test = test_env['exec_test']
#         except Exception as e:
#         # Catch syntax errors in model code or execution issues
#           return f"Failed at extraction!: {type(e).__name__}: {str(e)}"

#         for i in range(1):
#           test_input, expected_result = generate_test_case(i + 1)
#           test_env_2 = {"test_input": test_input}
#           print(test_input)
#           print(expected_result)
#           exec(clean_code, test_env_2)
#           assert exec_test(test_env["result"], expected_result)
#         return "Passed"

#     except AssertionError:
#         return "Failed: Logic Error (Assertion failed)"
#     except Exception as e:
#         # Catch syntax errors in model code or execution issues
#         return f"Failed at insertion!: {type(e).__name__}: {str(e)}"

- Given below is a custom testcase function where both the `inputs` and `expected output` is also extracted from the `code_context`

In [21]:
# def run_test(clean_code, code_context):
#     # 1. Environment for Oracle (Code 1 logic)
#     oracle_env = {"pd": pd, "np": np}
# #
#     try:
#         # STEP 1: Load the functions ONLY
#         code_context should contain the DEFS for generate_test_case and exec_test
#         try:
#             exec(code_context, oracle_env)
#             generate_test_case = oracle_env['generate_test_case']
#             exec_test = oracle_env['exec_test']
#         except Exception as e:
#             return f"Failed at extraction!: {type(e).__name__}: {str(e)}"

#         # STEP 2: The Loop
#         for i in range(1):
#             # Generate the specific test data for this iteration
#             test_input, expected_result = generate_test_case(i + 1)

#             # STEP 3: Setup Model Environment (Code 2)
#             # We must provide 'test_input' here so the model can unpack it
#             model_env = {
#                 "test_input": test_input,
#                 "pd": pd,
#                 "np": np
#             }

#             # STEP 4: Execution
#             # The model's code usually starts with: df, List = test_input
#             exec(clean_code, model_env)

#             # STEP 5: Validation
#             if "result" not in model_env:
#                 return "Failed: Model did not define 'result' variable"

#             # Use the oracle's test function to compare
#             if not exec_test(model_env["result"], expected_result):
#                 raise AssertionError("Model logic incorrect")

#         return "Passed"

#     except AssertionError:
#         return "Failed: Logic Error (Assertion failed)"
#     except Exception as e:
#         return f"Failed at insertion!: {type(e).__name__}: {str(e)}"

##### check on code_context

In [22]:
for i in range(10):
  print(extract_only_exec_context(df_ds1k['code_context'][i]))
  print('--------')

import pandas as pd
import numpy as np
df, List = test_input
--------
import pandas as pd
import numpy as np
df, List = test_input
--------
import pandas as pd
import numpy as np
df = test_input
--------
import pandas as pd
import numpy as np
df = test_input
--------
import pandas as pd
import numpy as np
def f(df):

df = test_input
result = f(df)
--------
import pandas as pd
import numpy as np
df = test_input
--------
import pandas as pd
import numpy as np
df = test_input
--------
import pandas as pd
import numpy as np
df = test_input
--------
import pandas as pd
import numpy as np
df = test_input
--------
import pandas as pd
import numpy as np
df = test_input
--------


In [23]:
for i in range(876,880):
  print(extract_only_exec_context(df_ds1k['code_context'][i]))
  print('--------')

import pandas as pd
import numpy as np
import sklearn
f = test_input

result = new_f
--------
import pandas as pd
import numpy as np
import sklearn
features = test_input

result = new_features
--------
import pandas as pd
import numpy as np
import sklearn
features = test_input
def solve(features):

new_features = solve(features)
result = new_features
--------
import pandas as pd
import numpy as np
import sklearn
features = test_input

result = new_features
--------


#### extract_python_code

In [24]:
def extract_python_code(text):
    """
    Extracts code from a markdown-style python block.
    If no block is found, it returns the original text
    (handling cases where the model forgets the tags).
    """
    # Pattern looks for ```python or ``` followed by code and ending with ```
    pattern = r"```(?:python)?\n?(.*?)```"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        return match.group(1).strip()
    return text.strip()


#clean_code = extract_python_code(response)

#### ds1k_test

In [25]:
def ds1k_test(clean_code, code_context):
    """
    Executes the code_context to load the official test_execution function,
    then runs the generated code through it.
    """
    # Create a sandbox environment
    test_env = {}

    try:

        exec(code_context, test_env)
        test_env['test_execution'](clean_code)

        return "Passed"

    except AssertionError:
        return "Failed: Logic Error (Assertion failed)"
    except Exception as e:
        # Catch syntax errors in model code or execution issues
        return f"Failed: {type(e).__name__}: {str(e)}"

#### ds1k_test_2

- here we print the clean code and full code

In [43]:
def ds1k_test_2(clean_code, code_context, print_code=True):
    """
    Executes the DS-1000 code context and prints the FULL code
    that actually gets executed (after insertion).
    """
    test_env = {}

    try:
        # Step 1: Load test_execution into environment
        exec(code_context, test_env)

        # Step 2: Manually reconstruct final executed code
        exec_context = test_env["exec_context"] if "exec_context" in test_env else None

        if exec_context is None:
            raise RuntimeError("exec_context not found in code_context")

        full_executed_code = exec_context.replace("[insert]", clean_code)

        # Step 3: Print or log it
        if print_code:
            print(full_executed_code)

        print('-------XXX-------')
        # Step 4: Run official test
        test_env["test_execution"](clean_code)

        return "Passed"

    except AssertionError:
        return "Failed: Logic Error (Assertion failed)"
    except Exception as e:
        return f"Failed: {type(e).__name__}: {str(e)}"


In [39]:
def ds1k_test_2(clean_code, code_context):
    test_env = {}

    full_code = None

    try:
        # Load context (defines exec_context + test_execution)
        exec(code_context, test_env)

        exec_context = test_env["exec_context"]
        full_code = exec_context.replace("[insert]", clean_code)

        # Run official DS-1000 test
        test_env["test_execution"](clean_code)

        return {
            "status": "Passed",
            "full_code": full_code
        }

    except AssertionError:
        return {
            "status": "Failed: Logic Error (Assertion)",
            "full_code": full_code
        }

    except Exception as e:
        return {
            "status": f"Failed: {type(e).__name__}: {str(e)}",
            "full_code": full_code
        }


### EXAMPLE

In [ ]:
# # 1. Generate
# formatted_messages = prompt_constuction(df_ds1k['prompt_2'][0],extract_only_exec_context(df_ds1k['code_context'][0]))
# inputs = tokenizer.apply_chat_template(
#     formatted_messages,
#     add_generation_prompt=True,
#     tokenize=True,
#     return_dict=True,
#     return_tensors="pt"
# ).to("cuda")

# with torch.no_grad():
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=512,
#         do_sample=False,
#         pad_token_id=tokenizer.eos_token_id
#     )


# gen_ids = outputs[0][len(inputs['input_ids'][0]):]
# raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
# clean_code = extract_python_code(raw_response)
# print(clean_code)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


import pandas as pd

# Given DataFrame
df = pd.DataFrame({
    'Col1': [1, 4, 7, 10, 13, 16],
    'Col2': [2, 5, 8, 11, 14, 17],
    'Col3': [3, 6, 9, 12, 15, 18],
    'Type': [1, 1, 2, 2, 3, 3]
})

# Given list
List = [2, 4, 0, 3, 1, 5]

# Create a mapping from Type to index
type_to_index = {type: idx for idx, type in enumerate(df['Type'].unique())}

# Map the list to the DataFrame's index
mapped_indices = [type_to_index[type] for type in List]

# Sort the DataFrame based on the mapped indices
result = df.iloc[mapped_indices]

# Display the result
print(result)


In [ ]:
# print(formatted_messages)

[{'role': 'system', 'content': 'You are an expert Data Scientist specializing in Python and library-specific tasks (Pandas, NumPy, Scikit-learn). Your output must be valid code only.'}, {'role': 'user', 'content': "### ENVIRONMENT CONTEXT\nThe following code is already defined in the environment. Do NOT redefine these variables:\n```python\nimport pandas as pd\nimport numpy as np\ndf, List = test_input\n```\n\n### PROBLEM SPECIFICATION\nProblem:\nI have the following DataFrame:\n    Col1  Col2  Col3  Type\n0      1     2     3     1\n1      4     5     6     1\n2      7     8     9     2\n3    10    11    12     2\n4    13    14    15     3\n5    16    17    18     3\n\n\nThe DataFrame is read from a CSV file. All rows which have Type 1 are on top, followed by the rows with Type 2, followed by the rows with Type 3, etc.\nI would like to shuffle the order of the DataFrame's rows according to a list. \\\nFor example, give a list [2, 4, 0, 3, 1, 5] and desired result should be:\n    Col1 

In [ ]:
# run_test(clean_code, extract_only_exec_context(df_ds1k['code_context'][0]))

"Failed at extraction!: NameError: name 'test_input' is not defined"

In [ ]:
# 1. Generate
formatted_messages = construct_ds1000_prompt_final(df_ds1k['prompt_2'][0],extract_only_exec_context(df_ds1k['code_context'][0]))
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )


gen_ids = outputs[0][len(inputs['input_ids'][0]):]
raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
clean_code = extract_python_code(raw_response)
print(clean_code)

result = df.iloc[List]


In [27]:
# run_test(clean_code, df_ds1k['code_context'][0])

NameError: name 'run_test' is not defined

In [28]:
print(extract_only_exec_context(df_ds1k['code_context'][876]))

import pandas as pd
import numpy as np
import sklearn
f = test_input

result = new_f


In [31]:
print(df_ds1k['prompt'][876])

Problem:

Given a list of variant length features, for example:

f = [
    ['t1'],
    ['t2', 't5', 't7'],
    ['t1', 't2', 't3', 't4', 't5'],
    ['t4', 't5', 't6']
]
where each sample has variant number of features and the feature dtype is str and already one hot.

In order to use feature selection utilities of sklearn, I have to convert the features to a 2D-array which looks like:

f
    t1  t2  t3  t4  t5  t6  t7
r1   1   0   0   0   0   0   0
r2   0   1   0   0   1   0   1
r3   1   1   1   1   1   0   0
r4   0   0   0   1   1   1   0
How could I achieve it via sklearn or numpy?

A:

<code>
import pandas as pd
import numpy as np
import sklearn
f = load_data()
</code>
new_f = ... # put solution in this variable
BEGIN SOLUTION
<code>



In [ ]:
# 1. Generate
formatted_messages = construct_ds1000_prompt_final(df_ds1k['prompt_2'][876],extract_only_exec_context(df_ds1k['code_context'][876]))
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )


gen_ids = outputs[0][len(inputs['input_ids'][0]):]
raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
clean_code = extract_python_code(raw_response)
print(clean_code)

from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
f_encoded = encoder.fit_transform(f)

# Convert the result to a DataFrame for better readability
result = pd.DataFrame(f_encoded, columns=encoder.get_feature_names_out())


In [ ]:
# 1. Generate
formatted_messages = construct_ds1000_prompt_final_v2(df_ds1k['prompt_2'][876],extract_only_exec_context(df_ds1k['code_context'][876]))
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )


gen_ids = outputs[0][len(inputs['input_ids'][0]):]
raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
clean_code = extract_python_code(raw_response)
print(clean_code)

from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the features
f_encoded = encoder.fit_transform(f)

# Convert the result to a DataFrame for better readability
result = pd.DataFrame(f_encoded, columns=encoder.get_feature_names_out())


In [50]:
print(extract_only_exec_context(df_ds1k['code_context'][876]))

import pandas as pd
import numpy as np
import sklearn
f = test_input

result = new_f


In [ ]:
# 1. Generate
formatted_messages = construct_ds1000_prompt_final_v3(df_ds1k['prompt_2'][876],extract_only_exec_context(df_ds1k['code_context'][876]))
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )


gen_ids = outputs[0][len(inputs['input_ids'][0]):]
raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
clean_code = extract_python_code(raw_response)
print(clean_code)

from sklearn.preprocessing import OneHotEncoder

# Assuming f is already defined in the LIVE ENVIRONMENT
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(f)

# Convert to DataFrame for easier manipulation
df_encoded = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out())

# Assign the result to the variable 'result'
result = df_encoded


In [ ]:
run_ds1000_official_test(clean_code, df_ds1k['code_context'][876])

"Failed: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'"

In [51]:
print(extract_only_exec_context(df_ds1k['code_context'][876]))

import pandas as pd
import numpy as np
import sklearn
f = test_input

result = new_f


In [ ]:
# print(run_test(clean_code, df_ds1k['code_context'][876]))

"Failed at insertion!: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'"

###### Range try

In [ ]:
for i in range(876,879):
    # 1. Generate
    formatted_messages = construct_ds1000_prompt_final(df_ds1k['prompt_2'][i],extract_only_exec_context(df_ds1k['code_context'][i]))
    inputs = tokenizer.apply_chat_template(
        formatted_messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )


    gen_ids = outputs[0][len(inputs['input_ids'][0]):]
    raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
    clean_code = extract_python_code(raw_response)
    print(i,'\n')
    print(clean_code)
    print(run_test(clean_code, df_ds1k['code_context'][i]))
    print('\n')
    print(run_ds1000_official_test(clean_code, df_ds1k['code_context'][i]))
    print('---------------')

876 

from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the data
f_encoded = encoder.fit_transform(f)

# Convert the result to a DataFrame for better readability
result = pd.DataFrame(f_encoded, columns=encoder.get_feature_names_out())
Failed at insertion!: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'


Failed: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'
---------------
877 

from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse=False)

# Fit and transform the features
result = encoder.fit_transform(features)
Failed at insertion!: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'


Failed: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'
---------------
878 

from sklearn.preprocessing import OneHotEncoder

# Ini

In [ ]:
for i in range(300,310):
    # 1. Generate
    formatted_messages = construct_ds1000_prompt_final(df_ds1k['prompt_2'][i],extract_only_exec_context(df_ds1k['code_context'][i]))
    inputs = tokenizer.apply_chat_template(
        formatted_messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )


    gen_ids = outputs[0][len(inputs['input_ids'][0]):]
    raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
    clean_code = extract_python_code(raw_response)
    print(i,'\n')
    print(clean_code)
    print(run_test(clean_code, df_ds1k['code_context'][i]))
    print('\n')
    print(run_ds1000_official_test(clean_code, df_ds1k['code_context'][i]))
    print('---------------')

300 

result = np.percentile(a, p)
Failed at insertion!: NameError: name 'a' is not defined


Passed
---------------
301 

import numpy as np

def vec2matrix(vec, ncol):
    return vec.reshape(-1, ncol)

# Assuming test_input is a tuple containing A and ncol
A, ncol = test_input

# Convert the 1D array A into a 2D array with ncol columns
result = vec2matrix(A, ncol)
Passed


Failed: NameError: name 'B' is not defined
---------------
302 

import numpy as np

def vec2matrix(vec, nrow):
    return vec.reshape(nrow, -1)

# Assuming test_input is a tuple containing the array and its number of rows
A, nrow = test_input

# Convert the 1D array to a 2D array with the specified number of rows
result = vec2matrix(A, nrow)
Passed


Failed: NameError: name 'B' is not defined
---------------
303 

import numpy as np

def vec2matrix(vec, ncol):
    # Calculate the number of rows needed
    nrow = len(vec) // ncol
    # If there's a remainder, add one more row
    if len(vec) % ncol != 0:
        nr

In [ ]:
for i in range(350,380):
    # 1. Generate
    formatted_messages = construct_ds1000_prompt_final(df_ds1k['prompt_2'][i],extract_only_exec_context(df_ds1k['code_context'][i]))
    inputs = tokenizer.apply_chat_template(
        formatted_messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )


    gen_ids = outputs[0][len(inputs['input_ids'][0]):]
    raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
    clean_code = extract_python_code(raw_response)
    print(i,'\n')
    print(clean_code)
    print(run_test(clean_code, df_ds1k['code_context'][i]))
    print('\n')
    print(run_ds1000_official_test(clean_code, df_ds1k['code_context'][i]))
    print('---------------')

350 

import numpy as np
from scipy.stats import ttest_ind_from_stats

# Extracting mean and standard deviation from the input
mean_a, std_a, n_a = a
mean_b, std_b, n_b = b

# Calculating the weighted t-statistic and p-value
result = ttest_ind_from_stats(mean1=mean_a, std1=std_a, nobs1=n_a,
                              mean2=mean_b, std2=std_b, nobs2=n_b,
                              equal_var=False)
Failed at insertion!: NameError: name 'a' is not defined


Failed: ValueError: too many values to unpack (expected 3)
---------------
351 

import numpy as np
from scipy.stats import ttest_ind_from_stats

# Assuming test_input is a tuple containing means and standard deviations of sample 1 and sample 2
mean1, std1, n1 = test_input[0]
mean2, std2, n2 = test_input[1]

# Calculate the weighted t-statistic and p-value
result = ttest_ind_from_stats(mean1=mean1, std1=std1, nobs1=n1,
                              mean2=mean2, std2=std2, nobs2=n2,
                              equal_var=False)



In [ ]:
for i in range(9):
    # 1. Generate
    formatted_messages = construct_ds1000_prompt_final(df_ds1k['prompt_2'][i],extract_only_exec_context(df_ds1k['code_context'][i]))
    inputs = tokenizer.apply_chat_template(
        formatted_messages,
        add_generation_prompt=True,
        tokenize=True,
        return_dict=True,
        return_tensors="pt"
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )


    gen_ids = outputs[0][len(inputs['input_ids'][0]):]
    raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
    clean_code = extract_python_code(raw_response)
    print(clean_code)
    print(run_test(clean_code, df_ds1k['code_context'][i]))
    print('\n')
    print(run_ds1000_official_test(clean_code, df_ds1k['code_context'][i]))
    print('---------------')

result = df.iloc[List]
Failed at insertion!: NameError: name 'df' is not defined


Passed
---------------
# Create a mapping from Type to index in the original DataFrame
type_to_index = {t: i for i, t in enumerate(df['Type'].unique())}

# Map the list to the original DataFrame's index
mapped_indices = [type_to_index[t] for t in List]

# Shuffle the DataFrame based on the mapped indices
result = df.iloc[mapped_indices]
Failed at insertion!: NameError: name 'df' is not defined


Failed: KeyError: np.int64(0)
---------------
# Create a dictionary to map counts to new values
value_counts_map = {
    True: 'other',
    False: lambda x: x if x in ['apple', 'egg'] else 'other'
}

# Apply the mapping to each column
result = df.applymap(lambda x: value_counts_map[bool(df[x].value_counts() >= 2)](x))
Failed at insertion!: NameError: name 'df' is not defined


Failed: KeyError: 'apple'
---------------


<string>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.


# Count occurrences of each value in each column
value_counts_qu1 = df['Qu1'].value_counts()
value_counts_qu2 = df['Qu2'].value_counts()
value_counts_qu3 = df['Qu3'].value_counts()

# Create a mask for values that appear 3 or more times
mask_qu1 = value_counts_qu1 >= 3
mask_qu2 = value_counts_qu2 >= 3
mask_qu3 = value_counts_qu3 >= 3

# Apply the mask to create the 'others' category
df['Qu1'] = df['Qu1'].where(mask_qu1, 'other')
df['Qu2'] = df['Qu2'].where(mask_qu2, 'other')
df['Qu3'] = df['Qu3'].where(mask_qu3, 'other')

# Rename the columns to match the test_data format
result = df.rename(columns={'Qu1': 'Qu1', 'Qu2': 'Qu2', 'Qu3': 'Qu3'})
Failed at insertion!: NameError: name 'df' is not defined


Failed: Logic Error (Assertion failed)
---------------
import pandas as pd

# Define the function to process the DataFrame
def f(df):
    # Create a dictionary to map counts to new values
    value_counts_dict = df.value_counts().to_dict()
    
    # Apply the mapping to each column
    df

##### ONE LAST TRY

In [ ]:
# 1. Generate
formatted_messages = construct_ds1000_prompt_final_x(df_ds1k['prompt_2'][5],extract_only_exec_context(df_ds1k['code_context'][5]))
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )


gen_ids = outputs[0][len(inputs['input_ids'][0]):]
raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
print(f'Task{5}')
clean_code = extract_python_code(raw_response)
print(clean_code,'\n')
print(run_ds1000_official_test(df_ds1k['prompt_2'][5],df_ds1k['code_context'][5]))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


NameError: name 'extract_python_code' is not defined

NameError: name 'extract_python_code' is not defined

##### NEW-TRY

In [26]:
# 1. Generate
formatted_messages = contruct_prompt_ds1k(df_ds1k['prompt_2'][876],extract_only_exec_context(df_ds1k['code_context'][876]))
inputs = tokenizer.apply_chat_template(
    formatted_messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt"
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )


gen_ids = outputs[0][len(inputs['input_ids'][0]):]
raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
print(f'Task{876}')
clean_code = extract_python_code(raw_response)
print(clean_code,'\n')
print(run_ds1000_official_test(df_ds1k['prompt_2'][876],df_ds1k['code_context'][876]))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Task876
from sklearn.preprocessing import OneHotEncoder

# Assuming f is already defined as per the task description
encoder = OneHotEncoder(sparse=False)
f_encoded = encoder.fit_transform(f)

# Convert the result to a DataFrame for easier manipulation
result = pd.DataFrame(f_encoded, columns=encoder.get_feature_names_out()) 

Failed: SyntaxError: invalid decimal literal (<string>, line 18)


In [33]:
print(ds1k_test(clean_code,df_ds1k['code_context'][876]))

Failed: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'


- just prints the  "FULL-CODE"

In [32]:
print(ds1k_test_2(clean_code,df_ds1k['code_context'][876]))


import pandas as pd
import numpy as np
import sklearn
f = test_input
from sklearn.preprocessing import OneHotEncoder

# Assuming f is already defined as per the task description
encoder = OneHotEncoder(sparse=False)
f_encoded = encoder.fit_transform(f)

# Convert the result to a DataFrame for easier manipulation
result = pd.DataFrame(f_encoded, columns=encoder.get_feature_names_out())
result = new_f

Failed: TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'


##### Let's try for 50 datas buddy!

- here what I found out is that result is conisdered already declared.
- thus created a new func `extract_only_ecec_context_wi` *(with insert marker)*

In [44]:
for i in range(25):
  # 1. Generate
  formatted_messages = contruct_prompt_ds1k(df_ds1k['prompt_2'][i],extract_only_exec_context(df_ds1k['code_context'][i]))
  inputs = tokenizer.apply_chat_template(
      formatted_messages,
      add_generation_prompt=True,
      tokenize=True,
      return_dict=True,
      return_tensors="pt"
  ).to("cuda")

  with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=512,
          do_sample=False,
          pad_token_id=tokenizer.eos_token_id
      )


  gen_ids = outputs[0][len(inputs['input_ids'][0]):]
  raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
  print(f'Task{i}')
  clean_code = extract_python_code(raw_response)
  print(clean_code,'\n')
  print('Full code:')
  print(ds1k_test_2(clean_code,df_ds1k['code_context'][i]))
  print('---------------------------------------')

Task0
result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------
Passed
---------------------------------------
Task1
result = len(df[df['Type'] != df['Type'].iloc[0]]) 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = len(df[df['Type'] != df['Type'].iloc[0]])

-------XXX-------
Passed
---------------------------------------
Task2
result = df.copy()
result['Qu1'] = np.where(df['Qu1'].value_counts() >= 2, df['Qu1'], 'other')
result['Qu3'] = np.where(df['Qu3'].value_counts() >= 2, df['Qu3'], 'other') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.copy()
result['Qu1'] = np.where(df['Qu1'].value_counts() >= 2, df['Qu1'], 'other')
result['Qu3'] = np.where(df['Qu3'].value_counts() >= 2, df['Qu3'], 'other')

-------XXX-------
Failed: ValueError: operands could not be broadcast together with shapes (5,) (9,) () 
-----------------------------------

<string>:10: SyntaxWarning: invalid escape sequence '\['
<string>:10: SyntaxWarning: invalid escape sequence '\]'


Task16
result.loc[result['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result.loc[result['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------
Failed: NameError: name 'result' is not defined
---------------------------------------
Task17
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task18
result.loc[(result['product'].isin([1069104, 1069105]) | result['product'].isin([1179159, 1179161])), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result.loc[(result['product'].isin([1069104, 1069105]) | result['product'].isin([117

- done with insert marker!

In [51]:
for i in range(25):
  # 1. Generate
  formatted_messages = contruct_prompt_ds1k(df_ds1k['prompt_2'][i],extract_only_exec_context_wi(df_ds1k['code_context'][i]))
  inputs = tokenizer.apply_chat_template(
      formatted_messages,
      add_generation_prompt=True,
      tokenize=True,
      return_dict=True,
      return_tensors="pt"
  ).to("cuda")

  with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=512,
          do_sample=False,
          pad_token_id=tokenizer.eos_token_id
      )


  gen_ids = outputs[0][len(inputs['input_ids'][0]):]
  raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
  print(f'Task{i}')
  clean_code = extract_python_code(raw_response)
  print(clean_code,'\n')
  print('Full code:')
  print(ds1k_test_2(clean_code,df_ds1k['code_context'][i]))
  print('---------------------------------------')

Task0
result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------
Passed
---------------------------------------
Task1
result = len(df[df['Type'] != df['Type'].iloc[0]]) 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = len(df[df['Type'] != df['Type'].iloc[0]])

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task2
result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x)

-------XXX-------
Failed: ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
---------------------------------------
Task3
result = df.copy()
result['Qu1'] = np.where(df['Qu1'].value_counts() >= 3, df['Qu1'], 'other')
result['Qu3'] = np.where(df['Qu3

<string>:10: SyntaxWarning: invalid escape sequence '\['
<string>:10: SyntaxWarning: invalid escape sequence '\]'


Task16
result.loc[result['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result.loc[result['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------
Failed: NameError: name 'result' is not defined
---------------------------------------
Task17
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task18
target_products = [[1069104, 1069105], [1179159, 1179161]]
constant = 10

# Create a mask for the target products
mask = df['product'].isin(target_products)

# Multiply the scores for the target products by the constant
df.loc[mask, 'score'] *= constant

result = df 

Full code:


- below is with **new prompt** `v2` which will make sure [insert] works.
- works really shitty.
- too long code.

In [54]:
for i in range(25):
  # 1. Generate
  formatted_messages = contruct_prompt_ds1k_v2(df_ds1k['prompt_2'][i],extract_only_exec_context_wi(df_ds1k['code_context'][i]))
  inputs = tokenizer.apply_chat_template(
      formatted_messages,
      add_generation_prompt=True,
      tokenize=True,
      return_dict=True,
      return_tensors="pt"
  ).to("cuda")

  with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=512,
          do_sample=False,
          pad_token_id=tokenizer.eos_token_id
      )


  gen_ids = outputs[0][len(inputs['input_ids'][0]):]
  raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
  print(f'Task{i}')
  clean_code = extract_python_code(raw_response)
  print(clean_code,'\n')
  print('Full code:')
  print(ds1k_test_2(clean_code,df_ds1k['code_context'][i]))
  print('---------------------------------------')

Task0
df = df.sort_values(by='Type')
df = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
df = df.sort_values(by='Type')
df = df.iloc[List]

-------XXX-------
Failed: KeyError: 'result'
---------------------------------------
Task1
# Define a function to shuffle the DataFrame based on the given list
def shuffle_dataframe(df, shuffle_list):
    # Create a mapping from Type to indices
    type_to_indices = {type: df[df['Type'] == type].index.tolist() for type in df['Type'].unique()}
    
    # Create a new list to store the shuffled indices
    shuffled_indices = []
    
    # Iterate over the shuffle list
    for index in shuffle_list:
        # Append the indices corresponding to the current Type
        shuffled_indices.extend(type_to_indices[index])
    
    # Shuffle the DataFrame using the shuffled indices
    df = df.iloc[shuffled_indices]
    
    return df

# Shuffle the DataFrame using the provided list
shuffled_df = shuffle_dataframe(df

<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Task4
df['Qu1'] = df['Qu1'].apply(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x) 

Full code:

import pandas as pd
import numpy as np
def f(df):
df['Qu1'] = df['Qu1'].apply(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x)
df = test_input
result = f(df)

-------XXX-------
Failed: IndentationError: expected an indented block after function definition on line 4 (<string>, line 5)
---------------------------------------
Task5
df['Qu1'] = df['Qu1'].apply(lambda x: 'other' if pd.value_counts(df['Qu1']) < 3 else x)
df['Qu2'] = df['Qu2'].apply(lambda x: x)
df['Qu3'] = df['Qu3'].apply(lambda x: 'other' if pd.value_counts(df['Qu3']) < 3 else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
df['Qu1'] = df['Qu1'].apply(lambda x: 'other' if pd.value_counts(df['Qu1']) < 3 else x)
df['Qu2'] = df['Qu2'].apply(lambda x: x)
df['Qu3'] = df['Qu3'].apply(lambda x: 'other' if pd.value_counts(df['Qu3']) < 3 else x)

-------XXX-------
Failed: Va

<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


KeyboardInterrupt: 

In [55]:
print(df_ds1k['code_context'][2])

import pandas as pd
import numpy as np
import copy


def generate_test_case(test_case_id):
    def generate_ans(data):
        df = data
        return df.where(df.apply(lambda x: x.map(x.value_counts())) >= 2, "other")

    def define_test_input(test_case_id):
        if test_case_id == 1:
            df = pd.DataFrame(
                {
                    "Qu1": [
                        "apple",
                        "potato",
                        "cheese",
                        "banana",
                        "cheese",
                        "banana",
                        "cheese",
                        "potato",
                        "egg",
                    ],
                    "Qu2": [
                        "sausage",
                        "banana",
                        "apple",
                        "apple",
                        "apple",
                        "sausage",
                        "banana",
                        "banana",
     

- `result` error
- indendation error from `v3`
- hence we are tryiny `v4`

In [58]:
for i in range(25):
  # 1. Generate
  formatted_messages = contruct_prompt_ds1k_v3(df_ds1k['prompt_2'][i],extract_only_exec_context_wi(df_ds1k['code_context'][i]))
  inputs = tokenizer.apply_chat_template(
      formatted_messages,
      add_generation_prompt=True,
      tokenize=True,
      return_dict=True,
      return_tensors="pt"
  ).to("cuda")

  with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=512,
          do_sample=False,
          pad_token_id=tokenizer.eos_token_id
      )


  gen_ids = outputs[0][len(inputs['input_ids'][0]):]
  raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
  print(f'Task{i}')
  clean_code = extract_python_code(raw_response)
  print(clean_code,'\n')
  print('Full code:')
  print(ds1k_test_2(clean_code,df_ds1k['code_context'][i]))
  print('---------------------------------------')

Task0
result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------
Passed
---------------------------------------
Task1
result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task2
df['Qu1'] = df['Qu1'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x)
df['Qu3'] = df['Qu3'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
df['Qu1'] = df['Qu1'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x)
df['Qu3'] = df['Qu3'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x)

-------XXX-------
Failed: KeyError: 'result'
---------------------------------------
Task3
result = df.apply(lambda x: 'other'

<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Task4
df['Qu1'] = df['Qu1'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x)
df['Qu3'] = df['Qu3'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x) 

Full code:

import pandas as pd
import numpy as np
def f(df):
df['Qu1'] = df['Qu1'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x)
df['Qu3'] = df['Qu3'].map(lambda x: 'other' if x not in ['cheese', 'potato', 'banana'] else x)
df = test_input
result = f(df)

-------XXX-------
Failed: IndentationError: expected an indented block after function definition on line 4 (<string>, line 5)
---------------------------------------
Task5
result = df.apply(lambda x: 'other' if x.isin(pd.value_counts(x) >= 3).any() else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.apply(lambda x: 'other' if x.isin(pd.value_counts(x) >= 3).any() else x)

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------


<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
<string>:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Task6
result = df.copy()
result['Qu1'] = np.where(df['Qu1'].value_counts() >= 3, df['Qu1'], 'other')
result['Qu2'] = np.where(df['Qu2'].value_counts() >= 2, df['Qu2'], 'other')
result['Qu3'] = np.where(df['Qu3'].value_counts() >= 2, df['Qu3'], 'other') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.copy()
result['Qu1'] = np.where(df['Qu1'].value_counts() >= 3, df['Qu1'], 'other')
result['Qu2'] = np.where(df['Qu2'].value_counts() >= 2, df['Qu2'], 'other')
result['Qu3'] = np.where(df['Qu3'].value_counts() >= 2, df['Qu3'], 'other')

-------XXX-------
Failed: ValueError: operands could not be broadcast together with shapes (5,) (9,) () 
---------------------------------------
Task7
result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first')

-------XXX-------
Failed: Logic Error (As

<string>:10: SyntaxWarning: invalid escape sequence '\['
<string>:10: SyntaxWarning: invalid escape sequence '\]'


Task16
result.loc[result['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result.loc[result['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------
Failed: NameError: name 'result' is not defined
---------------------------------------
Task17
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task18
result.loc[(df['product'].isin([1069104, 1069105]) | df['product'].isin([1179159, 1179161])), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result.loc[(df['product'].isin([1069104, 1069105]) | df['product'].isin([1179159, 1179161]))

<string>:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.


Task21
result['category'] = df.apply(lambda row: next((col for col in df.columns if row[col] == 1), None), axis=1) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result['category'] = df.apply(lambda row: next((col for col in df.columns if row[col] == 1), None), axis=1)
result = df

-------XXX-------
Failed: NameError: name 'result' is not defined
---------------------------------------
Task22
result['category'] = df.apply(lambda row: list(row.index[row == 1]), axis=1) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result['category'] = df.apply(lambda row: list(row.index[row == 1]), axis=1)
result = df

-------XXX-------
Failed: NameError: name 'result' is not defined
---------------------------------------
Task23
result['Date'] = df['Date'].dt.strftime('%b-%Y') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result['Date'] = df['Date'].dt.strftime('%b-%Y')
result = df

-------XXX-------
Failed: NameError: name 'result' is

In [60]:
for i in range(25):
  # 1. Generate
  formatted_messages = contruct_prompt_ds1k_v4(df_ds1k['prompt_2'][i],extract_only_exec_context_wi(df_ds1k['code_context'][i]))
  inputs = tokenizer.apply_chat_template(
      formatted_messages,
      add_generation_prompt=True,
      tokenize=True,
      return_dict=True,
      return_tensors="pt"
  ).to("cuda")

  with torch.no_grad():
      outputs = model.generate(
          **inputs,
          max_new_tokens=512,
          do_sample=False,
          pad_token_id=tokenizer.eos_token_id
      )


  gen_ids = outputs[0][len(inputs['input_ids'][0]):]
  raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
  print(f'Task{i}')
  clean_code = extract_python_code(raw_response)
  print(clean_code,'\n')
  print('Full code:')
  print(ds1k_test_2(clean_code,df_ds1k['code_context'][i]))
  print('---------------------------------------')

Task0
result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------
Passed
---------------------------------------
Task1
result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task2
result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x)

-------XXX-------
Failed: ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
---------------------------------------
Task3
result = df.apply(lambda x: 'other' if x.value_counts() >= 3 else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.apply(lambda x: 'other' if x.valu

<string>:7: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Task7
result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first')

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task8
result = df[df['drop_if_dup'] == 'No'].drop_duplicates(subset='url', keep='first') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df[df['drop_if_dup'] == 'No'].drop_duplicates(subset='url', keep='first')

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task9
result = df.sort_values(by=['url', 'keep_if_dup'], ascending=[True, False]).drop_duplicates(subset='url') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.sort_values(by=['url', 'keep_if_dup'], ascending=[True, False]).drop_duplicates(subset='url')

-------XXX-------
Failed: Logic 

<string>:10: SyntaxWarning: invalid escape sequence '\['
<string>:10: SyntaxWarning: invalid escape sequence '\]'


Task16
result = df[df['product'].isin([1069104, 1069105])].assign(score=lambda x: x['score'] * 10) 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df[df['product'].isin([1069104, 1069105])].assign(score=lambda x: x['score'] * 10)
result = df

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task17
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------
Failed: Logic Error (Assertion failed)
---------------------------------------
Task18
result = df[df['product'].isin([1069104, 1069105, 1179159, 1179161])].assign(score=lambda x: x['score'] * 10) 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df[df['product'].isin([1069104, 1069105, 117915


##### CSV_DOWNLOAD

In [35]:
from google.colab import files

In [37]:
def ds1k_evaluation_driver(
    dataframe,
    start_idx,
    end_idx,
    checkpoint_step=10
):
    """
    Driver for DS-1000 dataset with full executed code capture.
    """
    current_df = dataframe.iloc[start_idx:end_idx].copy()

    generated_codes = []
    full_exec_codes = []
    statuses = []

    print(f"🚀 Starting DS-1000 Batch: {start_idx} to {end_idx}")

    for i in range(len(current_df)):
        global_idx = start_idx + i
        row = current_df.iloc[i]

        # 1. Prompt construction
        formatted_messages = contruct_prompt_ds1k(
            row['prompt_2'],
            extract_only_exec_context(row['code_context'])
        )

        inputs = tokenizer.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cuda")

        # 2. Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        # 3. Decode
        gen_ids = outputs[0][len(inputs['input_ids'][0]):]
        raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
        clean_code = extract_python_code(raw_response)

        # 4. DS-1000 Testing (must return dict)
        result = ds1k_test_2(clean_code, row['code_context'])

        status = result["status"]
        full_code = result["full_code"]

        # 5. Store
        generated_codes.append(clean_code)
        full_exec_codes.append(full_code)
        statuses.append(status)

        print(
            f"✅ [{global_idx}] Task\n"
            f"STATUS: {status}\n"
            f"CLEAN CODE:\n{clean_code}\n"
        )

        # 6. Checkpoint
        if (i + 1) % checkpoint_step == 0:
            temp_df = current_df.head(len(generated_codes)).copy()
            temp_df["GENERATED_CODE"] = generated_codes
            temp_df["FULL_EXEC_CODE"] = full_exec_codes
            temp_df["STATUS"] = statuses

            ckpt_name = f"ds1k_checkpoint_{global_idx}.csv"
            temp_df.to_csv(ckpt_name, index=False)
            print(f"💾 Checkpoint saved: {ckpt_name}")

    # 7. Final save
    current_df["GENERATED_CODE"] = generated_codes
    current_df["FULL_EXEC_CODE"] = full_exec_codes
    current_df["STATUS"] = statuses

    filename = f"ds1k_results_{start_idx}_to_{end_idx}.csv"
    current_df.to_csv(filename, index=False)

    print(f"🏁 Batch Finished! Downloading {filename}")
    files.download(filename)

    return current_df


In [40]:
ds1k_evaluation_driver(df_ds1k,0,50)

🚀 Starting DS-1000 Batch: 0 to 50
✅ [0] Task
STATUS: Passed
CLEAN CODE:
result = df.iloc[List]

✅ [1] Task
STATUS: Passed
CLEAN CODE:
result = len(df[df['Type'] != df['Type'].iloc[0]])

✅ [2] Task
STATUS: Failed: ValueError: operands could not be broadcast together with shapes (5,) (9,) () 
CLEAN CODE:
result = df.copy()
result['Qu1'] = np.where(df['Qu1'].value_counts() >= 2, df['Qu1'], 'other')
result['Qu3'] = np.where(df['Qu3'].value_counts() >= 2, df['Qu3'], 'other')

✅ [3] Task
STATUS: Failed: ValueError: operands could not be broadcast together with shapes (5,) (9,) () 
CLEAN CODE:
result = df.copy()
result['Qu1'] = np.where(df['Qu1'].value_counts() >= 3, df['Qu1'], 'other')
result['Qu3'] = np.where(df['Qu3'].value_counts() >= 3, df['Qu3'], 'other')

✅ [4] Task
STATUS: Failed: IndentationError: expected an indented block after function definition on line 4 (<string>, line 5)
CLEAN CODE:
result = data.copy()
result['Qu1'] = data['Qu1'].map(lambda x: 'other' if x not in ['cheese', '

KeyboardInterrupt: 

In [61]:
def ds1k_test_2_m(clean_code, code_context, print_code=True):
    """
    Executes DS-1000 code context, prints full executed code,
    and RETURNS structured results for CSV logging.
    """
    test_env = {}
    full_executed_code = None

    try:
        # Load context (defines exec_context + test_execution)
        exec(code_context, test_env)

        exec_context = test_env.get("exec_context", None)
        if exec_context is None:
            raise RuntimeError("exec_context not found in code_context")

        # Build full executed code
        full_executed_code = exec_context.replace("[insert]", clean_code)

        # Print (keep your format)
        if print_code:
            print(full_executed_code)
            print("-------XXX-------")

        # Run official test
        test_env["test_execution"](clean_code)

        return {
            "status": "Passed",
            "full_code": full_executed_code
        }

    except AssertionError:
        return {
            "status": "Failed: Logic Error (Assertion failed)",
            "full_code": full_executed_code
        }

    except Exception as e:
        return {
            "status": f"Failed: {type(e).__name__}: {str(e)}",
            "full_code": full_executed_code
        }


In [67]:
def ds1k_simple_csv_driver_2(
    df_ds1k,
    start_idx,
    end_idx,
    filename=None
):
    """
    Runs DS-1000 generation from start_idx to end_idx and downloads CSV.

    CSV columns:
    - generated_code_snippet
    - full_code
    - Status
    """

    rows = []

    if filename is None:
        filename = f"ds1k_results_{start_idx}_to_{end_idx}.csv"

    print(f"🚀 Starting DS-1000 Batch: {start_idx} to {end_idx}")

    for i in range(start_idx, end_idx):
        # 1. Generate
        formatted_messages = contruct_prompt_ds1k_v4(
            df_ds1k['prompt_2'][i],
            extract_only_exec_context_wi(df_ds1k['code_context'][i])
        )

        inputs = tokenizer.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        # 2. Decode
        gen_ids = outputs[0][len(inputs['input_ids'][0]):]
        raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
        clean_code = extract_python_code(raw_response)

        # 🔹 KEEP YOUR PRINT FORMAT
        print(f"Task id: {i}")
        print("clean code: \n",clean_code, "\n")
        print("Full code:")

        # 3. Test
        result = ds1k_test_2_m(clean_code, df_ds1k['code_context'][i])

        rows.append({
            "generated_code_snippet": clean_code,
            "full_code": result["full_code"],
            "Status": result["status"]
        })
        print("\nSTATUS",result["status"])
        print("\n---------------------------------------")

    # 4. Save CSV
    result_df = pd.DataFrame(rows)
    result_df.to_csv(filename, index=False)

    # 5. Download
    try:
        from google.colab import files
        files.download(filename)
    except:
        print(f"Saved locally as {filename}")

    return result_df


In [68]:
ds1k_simple_csv_driver_2(df_ds1k,0,25)

🚀 Starting DS-1000 Batch: 0 to 25
Task id: 0
clean code: 
 result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------

STATUS Passed

---------------------------------------
Task id: 1
clean code: 
 result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 2
clean code: 
 result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x)

-------XXX-------

STATUS Failed: ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

---------------------------------------
Task id: 3
clean code: 
 result = df.apply(lambda x: 'other' if x.value

<string>:7: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Task id: 7
clean code: 
 result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first')

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 8
clean code: 
 result = df[df['drop_if_dup'] == 'No'].drop_duplicates(subset='url', keep='first') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df[df['drop_if_dup'] == 'No'].drop_duplicates(subset='url', keep='first')

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 9
clean code: 
 result = df.sort_values(by=['url', 'keep_if_dup'], ascending=[True, False]).drop_duplicates(subset='url') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.sort_values(by=['url', 'keep_if_dup'], ascending=[Tr

<string>:10: SyntaxWarning: invalid escape sequence '\['
<string>:10: SyntaxWarning: invalid escape sequence '\]'


Task id: 16
clean code: 
 result = df[df['product'].isin([1069104, 1069105])].assign(score=lambda x: x['score'] * 10) 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df[df['product'].isin([1069104, 1069105])].assign(score=lambda x: x['score'] * 10)
result = df

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 17
clean code: 
 result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 18
clean code: 
 result = df[df['product'].isin([1069104, 1069105, 1179159, 1179161])].assign(score=lambda x: x['score'] * 10) 

Full code:

import pandas as pd
import numpy as np
df, pro

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,generated_code_snippet,full_code,Status
0,result = df.iloc[List],"\nimport pandas as pd\nimport numpy as np\ndf,...",Passed
1,result = df.iloc[List],"\nimport pandas as pd\nimport numpy as np\ndf,...",Failed: Logic Error (Assertion failed)
2,result = df.apply(lambda x: 'other' if x.isin(...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: ValueError: The truth value of a Serie...
3,result = df.apply(lambda x: 'other' if x.value...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: ValueError: The truth value of a Serie...
4,df['Qu1'] = df['Qu1'].map(lambda x: 'other' if...,\nimport pandas as pd\nimport numpy as np\ndef...,Failed: IndentationError: expected an indented...
5,"result = df.replace(df.value_counts() >= 3, 'o...",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
6,result = df.copy()\nresult['Qu1'] = df['Qu1']....,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: ValueError: The truth value of a Serie...
7,result = df[df['keep_if_dup'] == 'Yes'].drop_d...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
8,result = df[df['drop_if_dup'] == 'No'].drop_du...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
9,"result = df.sort_values(by=['url', 'keep_if_du...",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)


In [73]:
def ds1k_simple_csv_driver_3(
    df_ds1k,
    start_idx,
    end_idx,
    filename=None
):
    """
    Runs DS-1000 generation for df_ds1k[start_idx:end_idx],
    appends results to THAT SLICE ONLY, and downloads it.
    """

    # Work on a slice copy only
    df_slice = df_ds1k.iloc[start_idx:end_idx].copy()

    if filename is None:
        filename = f"ds1k_results_{start_idx}_to_{end_idx}.csv"

    print(f"🚀 Starting DS-1000 Batch: {start_idx} to {end_idx}")

    # Ensure result columns exist
    for col in ["generated_code_snippet", "full_code", "Status"]:
        df_slice[col] = None

    for local_i, global_i in enumerate(range(start_idx, end_idx)):
        row = df_ds1k.iloc[global_i]

        # 1. Generate
        formatted_messages = contruct_prompt_ds1k_v4(
            row['prompt_2'],
            extract_only_exec_context_wi(row['code_context'])
        )

        inputs = tokenizer.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        # 2. Decode
        gen_ids = outputs[0][len(inputs['input_ids'][0]):]
        raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
        clean_code = extract_python_code(raw_response)

        # 🔹 KEEP YOUR PRINT FORMAT
        print(f"Task id: {global_i}")
        print("clean code: \n", clean_code, "\n")
        print("Full code:")

        # 3. Test
        result = ds1k_test_2_m(clean_code, row['code_context'])

        # 4. Write into slice dataframe
        df_slice.at[df_slice.index[local_i], "generated_code_snippet"] = clean_code
        df_slice.at[df_slice.index[local_i], "full_code"] = result["full_code"]
        df_slice.at[df_slice.index[local_i], "Status"] = result["status"]

        print("\nSTATUS", result["status"])
        print("\n---------------------------------------")

    # 5. Save SLICE only
    df_slice.to_csv(filename, index=False)

    # 6. Download
    try:
        from google.colab import files
        files.download(filename)
    except:
        print(f"Saved locally as {filename}")

    return df_slice


In [74]:
ds1k_simple_csv_driver_3(df_ds1k,25,27)

🚀 Starting DS-1000 Batch: 25 to 27
Task id: 25
clean code: 
 result['Date'] = df['Date'].dt.to_period("M").dt.strftime('%d-%b-%Y %A') 

Full code:

import pandas as pd
import numpy as np
df,List = test_input
result['Date'] = df['Date'].dt.to_period("M").dt.strftime('%d-%b-%Y %A')
result = df

-------XXX-------

STATUS Failed: NameError: name 'result' is not defined

---------------------------------------
Task id: 26
clean code: 
 result = df.shift(1, axis=0).shift(-1, axis=1) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.shift(1, axis=0).shift(-1, axis=1)
result = df

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,prompt,reference_code,metadata,code_context,prompt_2,generated_code_snippet,full_code,Status
25,Problem:\nI have the following DF\n\tDate\n0 ...,df = df[df['Date'] >= List[0]]\ndf = df[df['Da...,"{'problem_id': 25, 'library_problem_id': 25, '...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have the following DF\n\tDate\n0 ...,"result['Date'] = df['Date'].dt.to_period(""M"")....","\nimport pandas as pd\nimport numpy as np\ndf,...",Failed: NameError: name 'result' is not defined
26,Problem:\nSo I have a dataframe that looks lik...,import numpy as np\ndf['#1'] = np.roll(df['#1'...,"{'problem_id': 26, 'library_problem_id': 26, '...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nSo I have a dataframe that looks lik...,"result = df.shift(1, axis=0).shift(-1, axis=1)",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)


In [76]:
ds1k_simple_csv_driver_3(df_ds1k,0,100)

🚀 Starting DS-1000 Batch: 0 to 100
Task id: 0
clean code: 
 result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------

STATUS Passed

---------------------------------------
Task id: 1
clean code: 
 result = df.iloc[List] 

Full code:

import pandas as pd
import numpy as np
df, List = test_input
result = df.iloc[List]

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 2
clean code: 
 result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.apply(lambda x: 'other' if x.isin(['apple', 'egg']) else x)

-------XXX-------

STATUS Failed: ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

---------------------------------------
Task id: 3
clean code: 
 result = df.apply(lambda x: 'other' if x.valu

<string>:7: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.


Task id: 7
clean code: 
 result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df[df['keep_if_dup'] == 'Yes'].drop_duplicates(subset='url', keep='first')

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 8
clean code: 
 result = df[df['drop_if_dup'] == 'No'].drop_duplicates(subset='url', keep='first') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df[df['drop_if_dup'] == 'No'].drop_duplicates(subset='url', keep='first')

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 9
clean code: 
 result = df.sort_values(by=['url', 'keep_if_dup'], ascending=[True, False]).drop_duplicates(subset='url') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.sort_values(by=['url', 'keep_if_dup'], ascending=[Tr

<string>:10: SyntaxWarning: invalid escape sequence '\['
<string>:10: SyntaxWarning: invalid escape sequence '\]'


Task id: 16
clean code: 
 result = df[df['product'].isin([1069104, 1069105])].assign(score=lambda x: x['score'] * 10) 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df[df['product'].isin([1069104, 1069105])].assign(score=lambda x: x['score'] * 10)
result = df

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 17
clean code: 
 result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10 

Full code:

import pandas as pd
import numpy as np
df, products = test_input
result = df.copy()
result.loc[~df['product'].isin([1069104, 1069105]), 'score'] *= 10
result = df

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 18
clean code: 
 result = df[df['product'].isin([1069104, 1069105, 1179159, 1179161])].assign(score=lambda x: x['score'] * 10) 

Full code:

import pandas as pd
import numpy as np
df, pro

...

<string>:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`


Task id: 89
clean code: 
 result = np.where(np.all(df1[columns_check_list] == df2[columns_check_list], axis=1), False, True) 

Full code:

import pandas as pd
import numpy as np
df1, df2, columns_check_list = test_input
result = np.where(np.all(df1[columns_check_list] == df2[columns_check_list], axis=1), False, True)

-------XXX-------

STATUS Passed

---------------------------------------
Task id: 90
clean code: 
 result = np.all(df1[columns_check_list] == df2[columns_check_list], axis=1) 

Full code:

import pandas as pd
import numpy as np
df1, df2, columns_check_list = test_input
result = np.all(df1[columns_check_list] == df2[columns_check_list], axis=1)

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 91
clean code: 
 df.index.set_levels(pd.to_datetime(df.index.levels[1]), level=1) 

Full code:

import pandas as pd
import numpy as np
df = test_input
df.index.set_levels(pd.to_datetime(df.index.levels[1]), level=1)
r

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,prompt,reference_code,metadata,code_context,prompt_2,generated_code_snippet,full_code,Status
0,Problem:\nI have the following DataFrame:\n ...,"def g(df, List):\n return df.iloc[List]\n\n...","{'problem_id': 0, 'library_problem_id': 0, 'li...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have the following DataFrame:\n ...,result = df.iloc[List],"\nimport pandas as pd\nimport numpy as np\ndf,...",Passed
1,Problem:\nI have the following DataFrame:\n ...,"def g(df, List):\n df2 = df.iloc[List].rein...","{'problem_id': 1, 'library_problem_id': 1, 'li...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have the following DataFrame:\n ...,result = df.iloc[List],"\nimport pandas as pd\nimport numpy as np\ndf,...",Failed: Logic Error (Assertion failed)
2,Problem:\nI have following pandas dataframe :\...,def g(df):\n return df.where(df.apply(lambd...,"{'problem_id': 2, 'library_problem_id': 2, 'li...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have following pandas dataframe :\...,result = df.apply(lambda x: 'other' if x.isin(...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: ValueError: The truth value of a Serie...
3,Problem:\nI have following pandas dataframe :\...,def g(df):\n return df.where(df.apply(lambd...,"{'problem_id': 3, 'library_problem_id': 3, 'li...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have following pandas dataframe :\...,result = df.apply(lambda x: 'other' if x.value...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: ValueError: The truth value of a Serie...
4,Problem:\nI have following pandas dataframe :\...,result = df.where(df.apply(lambda x: x.map...,"{'problem_id': 4, 'library_problem_id': 4, 'li...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have following pandas dataframe :\...,df['Qu1'] = df['Qu1'].map(lambda x: 'other' if...,\nimport pandas as pd\nimport numpy as np\ndef...,Failed: IndentationError: expected an indented...
...,...,...,...,...,...,...,...,...
95,Problem:\nI have a data set which is in wide f...,def g(df):\n return df.set_index(['Country'...,"{'problem_id': 95, 'library_problem_id': 95, '...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a data set which is in wide f...,"result = df.melt(id_vars=['Country'], value_va...",\nimport pandas as pd\nimport numpy as np\ndf ...,"Failed: KeyError: ""The following id_vars or va..."
96,Problem:\nI have a data set which is in wide f...,def g(df):\n cols = list(df)[:2]+list(df)[-...,"{'problem_id': 96, 'library_problem_id': 96, '...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a data set which is in wide f...,"result = df.melt(id_vars=['Country'], value_va...",\nimport pandas as pd\nimport numpy as np\ndf ...,"Failed: KeyError: ""The following id_vars or va..."
97,Problem:\nI have a data frame like below \n ...,def g(df):\n mask = (df.filter(like='Value'...,"{'problem_id': 97, 'library_problem_id': 97, '...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a data frame like below \n ...,result = df[np.abs(df.filter(like='Value').val...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
98,Problem:\nI have a data frame like below \n ...,def g(df):\n mask = (df.filter(like='Value'...,"{'problem_id': 98, 'library_problem_id': 98, '...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a data frame like below \n ...,result = df[np.abs(df.filter(like='Value').val...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)


In [77]:
def ds1k_simple_csv_driver_4(
    df_ds1k,
    start_idx,
    end_idx,
    checkpoint_step=20,
    filename=None
):
    """
    Runs DS-1000 generation for df_ds1k[start_idx:end_idx],
    appends results to THAT SLICE ONLY,
    downloads checkpoint CSV every `checkpoint_step` tasks,
    and downloads final CSV.
    """

    # Work on slice only
    df_slice = df_ds1k.iloc[start_idx:end_idx].copy()

    if filename is None:
        filename = f"ds1k_results_{start_idx}_to_{end_idx}.csv"

    print(f"🚀 Starting DS-1000 Batch: {start_idx} to {end_idx}")

    # Ensure result columns exist
    for col in ["generated_code_snippet", "full_code", "Status"]:
        df_slice[col] = None

    for local_i, global_i in enumerate(range(start_idx, end_idx)):
        row = df_ds1k.iloc[global_i]

        # 1. Generate
        formatted_messages = contruct_prompt_ds1k_v4(
            row['prompt_2'],
            extract_only_exec_context_wi(row['code_context'])
        )

        inputs = tokenizer.apply_chat_template(
            formatted_messages,
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True,
            return_tensors="pt"
        ).to("cuda")

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id
            )

        # 2. Decode
        gen_ids = outputs[0][len(inputs['input_ids'][0]):]
        raw_response = tokenizer.decode(gen_ids, skip_special_tokens=True)
        clean_code = extract_python_code(raw_response)

        # 🔹 KEEP YOUR PRINT FORMAT
        print(f"Task id: {global_i}")
        print("clean code: \n", clean_code, "\n")
        print("Full code:")

        # 3. Test
        result = ds1k_test_2_m(clean_code, row['code_context'])

        # 4. Write into slice dataframe
        slice_idx = df_slice.index[local_i]
        df_slice.at[slice_idx, "generated_code_snippet"] = clean_code
        df_slice.at[slice_idx, "full_code"] = result["full_code"]
        df_slice.at[slice_idx, "Status"] = result["status"]

        print("\nSTATUS", result["status"])
        print("\n---------------------------------------")

        # 5. Checkpoint every `checkpoint_step` tasks
        if (local_i + 1) % checkpoint_step == 0:
            ckpt_end = start_idx + local_i + 1
            ckpt_name = f"ds1k_checkpoint_{start_idx}_to_{ckpt_end}.csv"

            df_slice.iloc[: local_i + 1].to_csv(ckpt_name, index=False)

            try:
                from google.colab import files
                files.download(ckpt_name)
            except:
                print(f"Checkpoint saved locally as {ckpt_name}")

    # 6. Final save
    df_slice.to_csv(filename, index=False)

    # 7. Final download
    try:
        from google.colab import files
        files.download(filename)
    except:
        print(f"Saved locally as {filename}")

    return df_slice


In [78]:
ds1k_simple_csv_driver_4(df_ds1k,100,200)

🚀 Starting DS-1000 Batch: 100 to 200
Task id: 100
clean code: 
 result = df.replace('&AMP;', '&') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.replace('&AMP;', '&')
result = df

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 101
clean code: 
 result = df.replace('&LT;', '<') 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.replace('&LT;', '<')
result = df

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------
Task id: 102
clean code: 
 df.replace('&AMP;', '&', regex=True, inplace=True) 

Full code:

import pandas as pd
import numpy as np
def f(df):
df.replace('&AMP;', '&', regex=True, inplace=True)
df = test_input
result = f(df)

-------XXX-------

STATUS Failed: IndentationError: expected an indented block after function definition on line 4 (<string>, line 5)

---------------------------------------

<string>:26: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


Task id: 118
clean code: 
 result = df.loc[df['alias'].isin(rows)] 

Full code:

import pandas as pd
import numpy as np
import io
df, test = test_input
result = df.loc[df['alias'].isin(rows)]

-------XXX-------

STATUS Failed: NameError: name 'rows' is not defined

---------------------------------------


<string>:25: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead


Task id: 119
clean code: 
 result = df.drop(df.index[df['alleles'].isin(test)]) 

Full code:

import pandas as pd
import numpy as np
import io
df, test = test_input
result = df.drop(df.index[df['alleles'].isin(test)])

-------XXX-------

STATUS Failed: Logic Error (Assertion failed)

---------------------------------------


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,prompt,reference_code,metadata,code_context,prompt_2,generated_code_snippet,full_code,Status
100,"Problem:\nIn pandas, how do I replace &AMP; wi...","def g(df):\n return df.replace('&AMP;','&',...","{'problem_id': 100, 'library_problem_id': 100,...",import pandas as pd\nimport numpy as np\nimpor...,"Problem:\nIn pandas, how do I replace &AMP; wi...","result = df.replace('&AMP;', '&')",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
101,"Problem:\nIn pandas, how do I replace &LT; wit...","def g(df):\n return df.replace('&LT;','<', ...","{'problem_id': 101, 'library_problem_id': 101,...",import pandas as pd\nimport numpy as np\nimpor...,"Problem:\nIn pandas, how do I replace &LT; wit...","result = df.replace('&LT;', '<')",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
102,"Problem:\nIn pandas, how do I replace &AMP; wi...","result = df.replace('&AMP;','&', regex=Tru...","{'problem_id': 102, 'library_problem_id': 102,...",import pandas as pd\nimport numpy as np\nimpor...,"Problem:\nIn pandas, how do I replace &AMP; wi...","df.replace('&AMP;', '&', regex=True, inplace=T...",\nimport pandas as pd\nimport numpy as np\ndef...,Failed: IndentationError: expected an indented...
103,"Problem:\nIn pandas, how do I replace &AMP;,&L...","def g(df):\n df.replace('&AMP;', '&', regex...","{'problem_id': 103, 'library_problem_id': 103,...",import pandas as pd\nimport numpy as np\nimpor...,"Problem:\nIn pandas, how do I replace &AMP;,&L...","result = df.replace('&AMP;', '&', regex=True)",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
104,"Problem:\nIn pandas, how do I replace &AMP; wi...",def g(df):\n for i in df.index:\n fo...,"{'problem_id': 104, 'library_problem_id': 104,...",import pandas as pd\nimport numpy as np\nimpor...,"Problem:\nIn pandas, how do I replace &AMP; wi...","result = df.replace('&AMP;', '&')",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)
...,...,...,...,...,...,...,...,...
195,Problem:\nWhat is an efficient way of splittin...,def g(df):\n return df.join(pd.DataFrame(df...,"{'problem_id': 195, 'library_problem_id': 195,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nWhat is an efficient way of splittin...,result = df.assign(var2=df['var2'].str.split('...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: TypeError: DataFrame.drop() takes from...
196,Problem:\nWhat is an efficient way of splittin...,def g(df):\n return df.join(pd.DataFrame(df...,"{'problem_id': 196, 'library_problem_id': 196,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nWhat is an efficient way of splittin...,result = df.assign(var2=df['var2'].str.split('...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: TypeError: DataFrame.drop() takes from...
197,Problem:\nI am trying to get count of special ...,"import numpy as np\ndef g(df):\n df[""new""] ...","{'problem_id': 197, 'library_problem_id': 197,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI am trying to get count of special ...,result = df['str'].apply(lambda x: sum(1 for c...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: AttributeError: module 'numpy' has no ...
198,Problem:\nI am trying to get count of letter c...,"def g(df):\n df[""new""] = df.apply(lambda p:...","{'problem_id': 198, 'library_problem_id': 198,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI am trying to get count of letter c...,result = df['str'].apply(lambda x: sum(1 for c...,\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: Logic Error (Assertion failed)


In [79]:
ds1k_simple_csv_driver_4(df_ds1k,200,300)

🚀 Starting DS-1000 Batch: 200 to 300
Task id: 200
clean code: 
 result = df.assign(fips=df['row'].str[:3], row=df['row'].str[3:]) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df.assign(fips=df['row'].str[:3], row=df['row'].str[3:])
result = df

-------XXX-------

STATUS Failed: TypeError: StringMethods.split() takes from 1 to 2 positional arguments but 3 were given

---------------------------------------
Task id: 201
clean code: 
 result = df['row'].str.split(',', expand=True)
result.columns = ['fips', 'medi', 'row'] 

Full code:

import pandas as pd
import numpy as np
df = test_input
result = df['row'].str.split(',', expand=True)
result.columns = ['fips', 'medi', 'row']
result = df

-------XXX-------

STATUS Failed: TypeError: StringMethods.split() takes from 1 to 2 positional arguments but 3 were given

---------------------------------------
Task id: 202
clean code: 
 result['cumulative_avg'] = df.iloc[:, 1:].apply(lambda x: x[x != 0].mean(), axis=1

<string>:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
<string>:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
<string>:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
<string>:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.75' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
<string>:17: FutureWarning: Setting an item of incompatible dty

Task id: 203
clean code: 
 result['cumulative_avg'] = df.iloc[:, :-1].apply(lambda x: x[x != 0].mean(), axis=1)
result.sort_index(axis=1, inplace=True) 

Full code:

import pandas as pd
import numpy as np
df = test_input
result['cumulative_avg'] = df.iloc[:, :-1].apply(lambda x: x[x != 0].mean(), axis=1)
result.sort_index(axis=1, inplace=True)
result = df

-------XXX-------

STATUS Failed: TypeError: can only concatenate str (not "int") to str

---------------------------------------


<string>:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
<string>:18: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


Task id: 204
clean code: 
 result = df.apply(lambda x: x[x != 0].mean(), axis=1) 

Full code:

import pandas as pd
import numpy as np
def f(df):
result = df.apply(lambda x: x[x != 0].mean(), axis=1)
df = test_input
result = f(df)

-------XXX-------

STATUS Failed: IndentationError: expected an indented block after function definition on line 4 (<string>, line 5)

---------------------------------------


...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,prompt,reference_code,metadata,code_context,prompt_2,generated_code_snippet,full_code,Status
200,Problem:\nI have a data frame with one (string...,def g(df):\n return pd.DataFrame(df.row.str...,"{'problem_id': 200, 'library_problem_id': 200,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a data frame with one (string...,"result = df.assign(fips=df['row'].str[:3], row...",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: TypeError: StringMethods.split() takes...
201,Problem:\nI have a data frame with one (string...,def g(df):\n return pd.DataFrame(df.row.str...,"{'problem_id': 201, 'library_problem_id': 201,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a data frame with one (string...,"result = df['row'].str.split(',', expand=True)...",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: TypeError: StringMethods.split() takes...
202,Problem:\nI have a Dataframe as below.\nName ...,def g(df):\n cols = list(df)[1:]\n for i...,"{'problem_id': 202, 'library_problem_id': 202,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a Dataframe as below.\nName ...,"result['cumulative_avg'] = df.iloc[:, 1:].appl...",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: NameError: name 'result' is not defined
203,Problem:\nI have a Dataframe as below.\nName ...,def g(df):\n cols = list(df)[1:]\n cols ...,"{'problem_id': 203, 'library_problem_id': 203,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a Dataframe as below.\nName ...,"result['cumulative_avg'] = df.iloc[:, :-1].app...",\nimport pandas as pd\nimport numpy as np\ndf ...,Failed: TypeError: can only concatenate str (n...
204,Problem:\nI have a Dataframe as below.\nName ...,cols = list(df)[1:]\n for idx in df.ind...,"{'problem_id': 204, 'library_problem_id': 204,...",import pandas as pd\nimport numpy as np\nimpor...,Problem:\nI have a Dataframe as below.\nName ...,"result = df.apply(lambda x: x[x != 0].mean(), ...",\nimport pandas as pd\nimport numpy as np\ndef...,Failed: IndentationError: expected an indented...
...,...,...,...,...,...,...,...,...
295,Problem:\nLet's say I have a 1d numpy positive...,"b = np.zeros((a.size, a.max()+1))\nb[np.arange...","{'problem_id': 295, 'library_problem_id': 4, '...",import numpy as np\nimport copy\nimport tokeni...,Problem:\nLet's say I have a 1d numpy positive...,b = np.eye(a.max() + 1)[a],\nimport numpy as np\na = test_input\nb = np.e...,Passed
296,Problem:\nLet's say I have a 1d numpy positive...,"b = np.zeros((a.size, a.max()+1))\nb[np.arange...","{'problem_id': 296, 'library_problem_id': 5, '...",import numpy as np\nimport copy\nimport tokeni...,Problem:\nLet's say I have a 1d numpy positive...,"b = np.eye(a.max() + 1)[a[:, None]]",\nimport numpy as np\na = test_input\nb = np.e...,Failed: Logic Error (Assertion failed)
297,Problem:\nLet's say I have a 1d numpy integer ...,"temp = a - a.min()\nb = np.zeros((a.size, temp...","{'problem_id': 297, 'library_problem_id': 6, '...",import numpy as np\nimport copy\nimport tokeni...,Problem:\nLet's say I have a 1d numpy integer ...,b = np.eye(np.max(a) - np.min(a) + 1)[a - np.m...,\nimport numpy as np\na = test_input\nb = np.e...,Passed
298,Problem:\nLet's say I have a 1d numpy array li...,"vals, idx = np.unique(a, return_inverse=True)\...","{'problem_id': 298, 'library_problem_id': 7, '...",import numpy as np\nimport copy\nimport tokeni...,Problem:\nLet's say I have a 1d numpy array li...,b = np.eye(len(np.unique(a))),\nimport numpy as np\na = test_input\nb = np.e...,Failed: Logic Error (Assertion failed)


In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,300,400)

In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,400,500)

In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,400,500)

In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,500,600)

In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,600,700)

In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,700,800)

In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,800,900)

In [ ]:
ds1k_simple_csv_driver_4(df_ds1k,900,1001)

## DS-1000

- What to do?
- Need to do 3 things,
  - generate code from model
  - run it against test and validate it.
  - Record this observation as csv.

## Problems

- The generated code will be tested using `code_context` column hwere inbuilt functions are defined like `test_execution`,our code is inserted in `[INSERT]` part of the `exec_context` varaible.

- check `prompt`,`code_context` column to get the understanding.
- we need 2 functions,
  - 1 good prompt constuction function
  - proper testing function!

- you can run the DATA LOADING columns to load data on `df_ds1k`
- `prompt_2` is contructed to avoid confusion.